ETL

In [56]:
#Se importan las librerias a usar
import pandas as pd
import numpy as np
import re 


Hoja de hechos

In [57]:
# Se importa el Dataset que se usara
df_hechos = pd.read_excel('../datasets/homicidios.xlsx', sheet_name='HECHOS')

In [58]:
df_hechos.head()# Visualiza las 5 primeras filas

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,NaN,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,...,2034.0,NaN,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO
3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,...,NaN,"VILLEGAS, CONRADO, GRAL.","LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,Point (99840.65224780 94269.16534422),-58.46503904,-34.68092974,MOTO-SD,MOTO,SD
4,2016-0005,1,2016-01-21,2016,1,21,05:20:00,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,...,NaN,"SAENZ PE?A, LUIS, PRES.","SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1,Point (106980.32827929 100752.16915795),-58.38718297,-34.62246630,MOTO-PASAJEROS,MOTO,PASAJEROS


In [59]:
df_hechos.info() # Da informacion de la columnas

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID                     696 non-null    object        
 1   N_VICTIMAS             696 non-null    int64         
 2   FECHA                  696 non-null    datetime64[ns]
 3   AAAA                   696 non-null    int64         
 4   MM                     696 non-null    int64         
 5   DD                     696 non-null    int64         
 6   HORA                   696 non-null    object        
 7   HH                     696 non-null    object        
 8   LUGAR_DEL_HECHO        696 non-null    object        
 9   TIPO_DE_CALLE          696 non-null    object        
 10  Calle                  695 non-null    object        
 11  Altura                 129 non-null    float64       
 12  Cruce                  525 non-null    object        
 13  Direc

Varias columnas tienen datos nulos, en el dicciononarios se evidencia que las filas sin datos contienen las palabra 'SD'

In [60]:
df_hechos = df_hechos.replace('SD', np.nan) # Reemplaza la palabra 'SD' por NaN

In [61]:
df_hechos.isnull().sum()# Cuenta los valores nulos por columna 

ID                         0
N_VICTIMAS                 0
FECHA                      0
AAAA                       0
MM                         0
DD                         0
HORA                       1
HH                         1
LUGAR_DEL_HECHO            1
TIPO_DE_CALLE              0
Calle                      1
Altura                   567
Cruce                    171
Dirección Normalizada      8
COMUNA                     0
XY (CABA)                  0
pos x                      0
pos y                      0
PARTICIPANTES              0
VICTIMA                    9
ACUSADO                   23
dtype: int64

Se verifica la primer columna con datos nulos

In [62]:
df_hechos[df_hechos['HORA'].isna()]

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
518,2019-0103,1,2019-12-18,2019,12,18,NaN,NaN,"PAZ, GRAL. AV. Y GRIVEO",GRAL PAZ,...,NaN,GRIVEO,"PAZ, GRAL. AV. y GRIVEO",11,Point (94643.11254058 103831.57115061),-58.52169422,-34.59471640,MOTO-MOTO,MOTO,MOTO


Como solo es una fila la que no contiene informacio podemos reemplazar por la moda de la columna

In [63]:
df_hechos.loc[df_hechos['HORA'].isna(), 'HORA'] = df_hechos['HORA'].mode().values[0]#Cambia el dato NaN por la primera moda

In [64]:
# Supongamos que 'df_hechos' es tu DataFrame y 'HORA' es la columna que deseas convertir
df_hechos['HORA'] = pd.to_datetime(df_hechos['HORA'], format='%H:%M:%S').dt.time

Se verifica la siguiente columna 'HH'

In [65]:
df_hechos[df_hechos['HH'].isna()]

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
518,2019-0103,1,2019-12-18,2019,12,18,14:00:00,NaN,"PAZ, GRAL. AV. Y GRIVEO",GRAL PAZ,...,NaN,GRIVEO,"PAZ, GRAL. AV. y GRIVEO",11,Point (94643.11254058 103831.57115061),-58.52169422,-34.59471640,MOTO-MOTO,MOTO,MOTO


En este caso se completa tomando la hora de la columna 'HORA' que es la misma moda calculada anteriormente

In [66]:
df_hechos.loc[518, 'HH'] = df_hechos.loc[518, 'HORA'].hour#Asigna la hora de la columna HORA a la coolumna  HH

In [67]:
df_hechos.loc[df_hechos.index == 518]# Revisa los pasos aplicados

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
518,2019-0103,1,2019-12-18,2019,12,18,14:00:00,14.0,"PAZ, GRAL. AV. Y GRIVEO",GRAL PAZ,...,NaN,GRIVEO,"PAZ, GRAL. AV. y GRIVEO",11,Point (94643.11254058 103831.57115061),-58.52169422,-34.59471640,MOTO-MOTO,MOTO,MOTO


Reviso la columna lugar de los hechos

In [68]:
df_hechos[df_hechos['LUGAR_DEL_HECHO'].isnull()]

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
119,2016-0151,1,2016-11-18,2016,11,18,20:35:00,20.0,NaN,CALLE,...,NaN,NaN,NaN,0,Point (. .),.,.,PEATON-SD,PEATON,NaN


Se nota la presencia de valores nulos en las columnas 'calle', 'altura', 'cruce' y 'dirección normalizada'. Es interesante destacar que las columnas 'dirección normalizada' y 'Lugar del hecho' parecen ser una fusión de 'calle' y 'cruce', lo que implica que algunos registros contienen información solo en 'calle' o exclusivamente en 'cruce', generando cierta inconsistencia en los datos. Por otra parte, la columna 'altura' exhibe una cantidad considerable de valores nulos en comparación con el conjunto total de datos.

Con el objetivo de mejorar la confiabilidad y la integridad de la información de ubicación, se optará por emplear las columnas 'XY (CABA)' y las coordenadas de longitud y latitud. Dada la falta de consistencia en los datos de 'calle', 'altura', 'cruce' y 'dirección normalizada', se tomará la decisión de descartar estas columnas durante el análisis.

Esta estrategia se traducirá en una mejora significativa de la calidad y la integridad de la información utilizada para la ubicación, contribuyendo así a la robustez de los análisis subsiguiente

In [69]:
df_hechos.drop(columns={'LUGAR_DEL_HECHO', 'Calle','Altura','Cruce','Dirección Normalizada'}, axis=1, inplace=True)# Elimina las columnas descartadas

Se verifican las columnas que se usaran para la ubicacion

In [70]:
df_hechos['pos y'].value_counts()

pos y
.               12
-34.68475866     5
-34.54979510     4
-34.53476874     4
-34.66977709     4
                ..
-34.58498660     1
-34.59573848     1
-34.63997430     1
-34.64234110     1
-34.58679619     1
Name: count, Length: 605, dtype: int64

In [71]:
df_hechos['pos y'].value_counts()

pos y
.               12
-34.68475866     5
-34.54979510     4
-34.53476874     4
-34.66977709     4
                ..
-34.58498660     1
-34.59573848     1
-34.63997430     1
-34.64234110     1
-34.58679619     1
Name: count, Length: 605, dtype: int64

Se evidencia un valor no numerico se verifica si es el mismo para las dos

In [72]:
df_hechos[(df_hechos['pos x']=='.') & (df_hechos['pos y']=='.')]# Filtra por '.' en las columnas pos x y pos y

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,TIPO_DE_CALLE,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
38,2016-0052,1,2016-04-20,2016,4,20,20:00:00,20.0,AUTOPISTA,13,Point (. .),.,.,MOTO-SD,MOTO,NaN
106,2016-0136,1,2016-10-25,2016,10,25,00:00:00,0.0,AUTOPISTA,4,Point (. .),.,.,MOTO-CARGAS,MOTO,CARGAS
119,2016-0151,1,2016-11-18,2016,11,18,20:35:00,20.0,CALLE,0,Point (. .),.,.,PEATON-SD,PEATON,NaN
139,2016-0174,1,2016-12-27,2016,12,27,00:00:00,0.0,AUTOPISTA,0,Point (. .),.,.,SD-SD,NaN,NaN
176,2017-0042,1,2017-04-10,2017,4,10,09:00:00,9.0,GRAL PAZ,14,Point (. .),.,.,MOTO-CARGAS,MOTO,CARGAS
180,2017-0050,2,2017-04-28,2017,4,28,11:08:08,11.0,AUTOPISTA,9,Point (. .),.,.,MOTO-CARGAS,MOTO,CARGAS
181,2017-0051,1,2017-05-01,2017,5,1,03:47:47,3.0,AUTOPISTA,7,Point (. .),.,.,AUTO-AUTO,AUTO,AUTO
256,2017-0140,1,2017-11-19,2017,11,19,23:22:17,23.0,AUTOPISTA,4,Point (. .),.,.,MOTO-PASAJEROS,MOTO,PASAJEROS
313,2018-0039,1,2018-04-21,2018,4,21,22:15:00,22.0,AUTOPISTA,14,Point (. .),.,.,PEATON-AUTO,PEATON,AUTO
546,2020-0026,1,2020-05-17,2020,5,17,06:40:00,6.0,AUTOPISTA,14,Point (. .),.,.,MOTO-OBJETO FIJO,MOTO,OBJETO FIJO


Dado que los hechos ocurrieron en la misma ciudad, se puede usar datos estadisticos para calcular un posible punto 'x' y 'y' de acuerdo a los participantes, para ello se procede primero a limpiar las columnas VICTIMA y ACUSADO

In [73]:
df_hechos[(df_hechos['VICTIMA'].isnull())&(df_hechos['ACUSADO'].isnull())]

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,TIPO_DE_CALLE,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
35,2016-0049,1,2016-04-17,2016,4,17,00:00:00,0.0,AUTOPISTA,4,Point (. .),-58.37714647568196,-34.63657525428238,SD-SD,NaN,NaN
76,2016-0101,1,2016-08-07,2016,8,7,19:42:00,19.0,GRAL PAZ,9,Point (93946.81673714 97681.68907836),-58.52932872,-34.65014993,SD-SD,NaN,NaN
92,2016-0119,1,2016-09-04,2016,9,4,06:40:00,6.0,AVENIDA,12,Point (96944.69676081 107291.52292157),-58.49659234,-34.56353635,SD-SD,NaN,NaN
139,2016-0174,1,2016-12-27,2016,12,27,00:00:00,0.0,AUTOPISTA,0,Point (. .),.,.,SD-SD,NaN,NaN
201,2017-0074,1,2017-06-04,2017,6,4,12:50:00,12.0,CALLE,9,Point (94281.31244858 99043.29027489),-58.52567073,-34.63787791,SD-SD,NaN,NaN


Se encontro un dato que parece ser un error en la comuna cero.

In [74]:
df_hechos[df_hechos['COMUNA']==0]

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,TIPO_DE_CALLE,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
119,2016-0151,1,2016-11-18,2016,11,18,20:35:00,20.0,CALLE,0,Point (. .),.,.,PEATON-SD,PEATON,NaN
139,2016-0174,1,2016-12-27,2016,12,27,00:00:00,0.0,AUTOPISTA,0,Point (. .),.,.,SD-SD,NaN,NaN


Considerando que solo existen dos filas asociadas a la comuna 0, se explorarán métodos estadísticos para inferir la ubicación de los incidentes. No obstante, se procederá a eliminar la fila 139, ya que únicamente contiene información acerca de una víctima y su fecha, careciendo de otros detalles relevantes. Dada la escasa cantidad de filas con datos faltantes, se plantea la posibilidad de realizar un análisis estadístico con el fin de sustituir estos valores. Se llevará a cabo el cálculo de la moda por comuna para reemplazar los datos faltantes en aquellas filas donde ninguna de las dos columnas posea información, utilizando el valor modal como sustituto. Este enfoque permitirá mejorar la integridad y la coherencia de los datos, contribuyendo a un análisis más preciso y completo.

In [75]:
df_hechos = df_hechos.drop(139, inplace=False)#Elimina la fila 119 

In [76]:
df_hechos['COMUNA'] = df_hechos['COMUNA'].replace(0, df_hechos['COMUNA'].mode()[0])# Se asigna la moda de las comunas a la comuna 0

In [77]:
# Calcular las modas por comuna 
moda_por_comuna = df_hechos.groupby('COMUNA')[['VICTIMA', 'ACUSADO']].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan)

In [78]:
# Reemplazar los valores nulos en 'ACUSADO' por las modas correspondientes por comuna 
df_hechos['ACUSADO'] = df_hechos.apply(lambda row: moda_por_comuna.loc[row['COMUNA'], 'ACUSADO'] if pd.isnull(row['ACUSADO']) and pd.isnull(row['VICTIMA']) else row['ACUSADO'], axis=1)


Ahora, estamos calculando la moda para cada valor único en cada columna. La finalidad es identificar la moda de la columna de víctimas cuando se presenta un acusado, y viceversa.

In [79]:
df_hechos_filtrado = df_hechos[(df_hechos['VICTIMA'].notnull()) & (df_hechos['ACUSADO'].notnull())]#Crea un nuevo df para no alterar el original

In [80]:
#Calcula las modas para cada columna
moda_victima_acusado = df_hechos_filtrado.groupby('ACUSADO')['VICTIMA'].agg(lambda x: x.mode().iloc[0] if x.mode().any() else None).to_dict()
moda_acusado_victima = df_hechos_filtrado.groupby('VICTIMA')['ACUSADO'].agg(lambda x: x.mode().iloc[0] if x.mode().any() else None).to_dict()

In [81]:
# Reemplaza los valores 'SD' en 'VICTIMA' por las modas correspondientes
df_hechos['VICTIMA'] = df_hechos['VICTIMA'].fillna(df_hechos['ACUSADO'].map(moda_victima_acusado))
# Reemplaza los valores 'SD' en 'ACUSADO' por las modas correspondientes
df_hechos['ACUSADO'] = df_hechos['ACUSADO'].fillna(df_hechos['VICTIMA'].map(moda_acusado_victima))

In [82]:
# Reemplaza los valores null en 'VICTIMA' por las modas correspondientes por comuna
df_hechos['VICTIMA'] = df_hechos.apply(lambda row: moda_por_comuna.loc[row['COMUNA'], 'VICTIMA'] if pd.isnull(row['VICTIMA']) else row['VICTIMA'], axis=1)

# Reemplaza los valores null en 'ACUSADO' por las modas correspondientes por comuna
df_hechos['ACUSADO'] = df_hechos.apply(lambda row: moda_por_comuna.loc[row['COMUNA'], 'ACUSADO'] if pd.isnull(row['ACUSADO']) else row['ACUSADO'], axis=1)


In [83]:
df_hechos['PARTICIPANTES'] = df_hechos['VICTIMA'] + '-' + df_hechos['ACUSADO']# Actualiza la columna PARTICIPANTES

Despues de realizar este procedimiento se puede continualr con las columnas por x y pos y

In [84]:
# Crea un nuevo df con la filas que contienen informacion en pos x y pos y
df_hechos_fil = df_hechos[(df_hechos['pos x'] != '.') & (df_hechos['pos y'] != '.')]

In [85]:
# Crea una nueva columna concatenando 'pos x' y 'pos y'
df_hechos_fil['coord_concat'] = df_hechos_fil['pos x'].astype(str) + '_' + df_hechos['pos y'].astype(str)

C:\Users\Willc\AppData\Local\Temp\ipykernel_19776\1879428845.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_hechos_fil['coord_concat'] = df_hechos_fil['pos x'].astype(str) + '_' + df_hechos['pos y'].astype(str)


In [86]:
# Calcula la moda de la concatenación de coordenadas para cada comuna
moda_coor_por_comuna = df_hechos_fil.groupby('COMUNA')['coord_concat'].agg(lambda x: x.mode().iloc[0] if x.mode().any() else None)

In [87]:
# Itera sobre las modas calculadas por comuna
for comuna, moda_concat in moda_coor_por_comuna.items():
    # Separa la moda de la concatenación en 'pos x' y 'pos y'
    moda_x, moda_y = moda_concat.split('_')
    
    # Reemplaza los valores faltantes en 'pos x' y 'pos y' con las modas correspondientes
    mascara = df_hechos['COMUNA'] == comuna
    df_hechos.loc[mascara, 'pos x'] = df_hechos.loc[mascara, 'pos x'].replace('.', moda_x)
    df_hechos.loc[mascara, 'pos y'] = df_hechos.loc[mascara, 'pos y'].replace('.', moda_y)


Con este procedimiento se reemplazan los valor nullos de pos x y pos y con moda de acuerdo a la comuna, esto quiere decir las ubicacion por comuna donde ocurren mas accidentes

In [88]:
# Convierte las columnas  'pos x' y 'pos y' a tipo de dato float
df_hechos['pos x'] = df_hechos['pos x'].astype(float) 
df_hechos['pos y'] = df_hechos['pos y'].astype(float) 

In [89]:
df_hechos.rename(columns={'pos x': 'LONGITUD', 'pos y': 'LATITUD'}, inplace=True)# Se cambia el nombre de las columnas pos x y pos y

verificamos valores nulos

In [90]:
df_hechos.isnull().sum()

ID               0
N_VICTIMAS       0
FECHA            0
AAAA             0
MM               0
DD               0
HORA             0
HH               0
TIPO_DE_CALLE    0
COMUNA           0
XY (CABA)        0
LONGITUD         0
LATITUD          0
PARTICIPANTES    0
VICTIMA          0
ACUSADO          0
dtype: int64

La columna "XY (CABA)" se relaciona con la geocodificación plana, mientras que las columnas de latitud (pos y) y longitud (pos x) contienen coordenadas que representan ubicaciones geográficas precisas. En consecuencia, podemos prescindir de la columna "XY (CABA)".

In [91]:
df_hechos.drop('XY (CABA)', axis=1, inplace=True)# Se elimina la columna  XY (CABA)

Se verifican datos duplicados

In [92]:
df_hechos[df_hechos.duplicated()]

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,TIPO_DE_CALLE,COMUNA,LONGITUD,LATITUD,PARTICIPANTES,VICTIMA,ACUSADO


Se exportal el df a un csv

In [93]:
df_hechos.to_csv('../datasets/homicidios_clean.csv', index=False)

Se importa la Hoja de Victimas

In [94]:
#importamos las hojas del Dataset que se usaran 
df_victimas = pd.read_excel('../Datasets/homicidios.xlsx', sheet_name='VICTIMAS')

In [95]:
df_victimas.head()# Visualiza las 5 primeras filas

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,2016-01-01,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01 00:00:00
1,2016-0002,2016-01-02,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70,2016-01-02 00:00:00
2,2016-0003,2016-01-03,2016,1,3,CONDUCTOR,MOTO,MASCULINO,30,2016-01-03 00:00:00
3,2016-0004,2016-01-10,2016,1,10,CONDUCTOR,MOTO,MASCULINO,18,SD
4,2016-0005,2016-01-21,2016,1,21,CONDUCTOR,MOTO,MASCULINO,29,2016-02-01 00:00:00


In [96]:
df_victimas.info()#  Descripción concisa y resumida del DataFrame.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID_hecho             717 non-null    object        
 1   FECHA                717 non-null    datetime64[ns]
 2   AAAA                 717 non-null    int64         
 3   MM                   717 non-null    int64         
 4   DD                   717 non-null    int64         
 5   ROL                  717 non-null    object        
 6   VICTIMA              717 non-null    object        
 7   SEXO                 717 non-null    object        
 8   EDAD                 717 non-null    object        
 9   FECHA_FALLECIMIENTO  717 non-null    object        
dtypes: datetime64[ns](1), int64(3), object(6)
memory usage: 56.1+ KB


In [97]:
df_victimas[df_victimas.duplicated()]# Verifica duplicados

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO


In [98]:
df_victimas[df_victimas['ID_hecho'].duplicated()]

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
30,2016-0041,2016-03-29,2016,3,29,PASAJERO_ACOMPAÑANTE,MOTO,MASCULINO,SD,2016-03-30 00:00:00
99,2016-0126,2016-09-18,2016,9,18,PASAJERO_ACOMPAÑANTE,AUTO,MASCULINO,60,SD
164,2017-0026,2017-02-26,2017,2,26,CONDUCTOR,AUTO,MASCULINO,19,2017-02-26 00:00:00
174,2017-0035,2017-03-23,2017,3,23,PASAJERO_ACOMPAÑANTE,AUTO,MASCULINO,32,2017-03-23 00:00:00
175,2017-0035,2017-03-23,2017,3,23,PASAJERO_ACOMPAÑANTE,AUTO,MASCULINO,30,2017-03-23 00:00:00
177,2017-0036,2017-03-29,2017,3,29,CONDUCTOR,MOTO,MASCULINO,20,2017-03-29 00:00:00
187,2017-0050,2017-04-28,2017,4,28,PASAJERO_ACOMPAÑANTE,MOTO,MASCULINO,16,2017-04-28 00:00:00
238,2017-0108,2017-09-02,2017,9,2,PASAJERO_ACOMPAÑANTE,AUTO,MASCULINO,10,2017-09-02 00:00:00
243,2017-0112,2017-09-10,2017,9,10,PASAJERO_ACOMPAÑANTE,AUTO,MASCULINO,SD,2017-09-13 00:00:00
254,2017-0126,2017-10-14,2017,10,14,CONDUCTOR,AUTO,MASCULINO,36,2017-10-14 00:00:00


La presencia de duplicados en la columna ID_Hecho no constituye un error, ya que cada fila representa a una víctima en cada accidente.

En el diccionario se evevidencias que las filas con datos nullos contiene la pala 'SD'

In [99]:
df_victimas = df_victimas.replace('SD', np.nan)# Reemplaza los valores 'SD' por NaN

Se verifican los datos nulos

In [100]:
df_victimas.isnull().sum()

ID_hecho                0
FECHA                   0
AAAA                    0
MM                      0
DD                      0
ROL                    11
VICTIMA                 9
SEXO                    6
EDAD                   53
FECHA_FALLECIMIENTO    68
dtype: int64

Se empieza reemplazando los datos de las victimas con los datos ya procesados de df_hechos

In [101]:
# Identificar las filas con valores faltantes en df_victimas
filas_faltantes = df_victimas[df_victimas['VICTIMA'].isnull()]

In [102]:
# Convertir el índice de df_victimas en una columna
filas_faltantes.reset_index(inplace=True)

In [103]:
# Completar los valores faltantes usando merge
df_victimas_completado = filas_faltantes.merge(df_hechos[['ID', 'VICTIMA']], left_on='ID_hecho', right_on='ID', how='left')
# Eliminar la columna redundante 'ID' resultante del merge
df_victimas_completado.drop(columns={'ID', 'VICTIMA_x'}, axis=1, inplace=True)


In [104]:
# Renombrar la columna 'VICTIMA_y' a 'VICTIMA'
df_victimas_completado.rename(columns={'VICTIMA_y': 'VICTIMA'}, inplace=True)

In [105]:
# Convierte la columna index en el indicice del df
df_victimas_completado.set_index('index', inplace=True)
# Actualizar df_victimas con los valores completados
df_victimas.update(df_victimas_completado)

In [106]:
df_victimas[df_victimas['VICTIMA'].isnull()]

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
141,2016-0174,2016-12-27,2016,12,27,NaN,NaN,NaN,NaN,NaN


Esta fila hace referencia a la que fue eliminada previamente en df_hechos. Al carecer de información significativa, se procederá a su eliminación.

In [107]:
# Elimina la fila con índice 141
df_victimas.drop(141, axis=0, inplace=True)

Se verifica la columna ROL

In [108]:
df_victimas[df_victimas['ROL'].isnull()]# Identifica las filas con valores faltantes

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
36,2016-0049,2016-04-17,2016,4,17,NaN,MOTO,NaN,NaN,NaN
39,2016-0052,2016-04-20,2016,4,20,NaN,MOTO,NaN,NaN,NaN
63,2016-0085,2016-06-29,2016,6,29,NaN,MOTO,MASCULINO,NaN,NaN
77,2016-0101,2016-08-07,2016,8,7,NaN,MOTO,MASCULINO,67.0,NaN
89,2016-0115,2016-09-02,2016,9,2,NaN,MOTO,MASCULINO,NaN,NaN
167,2017-0029,2017-03-07,2017,3,7,NaN,MOTO,MASCULINO,34.0,2017-03-07 00:00:00
208,2017-0074,2017-06-04,2017,6,4,NaN,MOTO,MASCULINO,70.0,2017-06-04 00:00:00
221,2017-0089,2017-07-13,2017,7,13,NaN,MOTO,MASCULINO,23.0,NaN
280,2017-0155,2017-12-12,2017,12,12,NaN,PEATON,MASCULINO,77.0,NaN
390,2018-0100,2018-09-20,2018,9,20,NaN,MOTO,FEMENINO,33.0,2018-09-20 00:00:00


In [109]:
df_victimas_rol = df_victimas[df_victimas['ROL'].notnull()]# Se carga un nuevo df con  datos not null

In [110]:
moda_rol = df_victimas_rol .groupby('VICTIMA')['ROL'].agg(lambda x: x.mode().iloc[0])# Calcular la moda

In [111]:
# Reemplazar los valores null en 'ROL' con la moda correspondiente de 'VICTIMA'
for index, row in df_victimas.iterrows():
    # Verificar si 'ROL' es nulo y 'VICTIMA' está presente en el índice de moda_rol
    if pd.isnull(row['ROL']) and row['VICTIMA'] in moda_rol.index:
        # Asignar la moda correspondiente a 'ROL'
        df_victimas.at[index, 'ROL'] = moda_rol[row['VICTIMA']]


Dado que solo contamos con 53 datos sin información de edad de un total de 717, sería factible calcular el promedio de edad de las víctimas según el tipo de vehículo.

In [112]:
# Filtra las filas notnull
df_victimas_cl = df_victimas[df_victimas['EDAD'].notnull()]

In [113]:
# Calcula la media de edad para cada tipo de víctima
media_edad_victimas = df_victimas_cl.groupby('VICTIMA')['EDAD'].mean().reset_index()

In [114]:
# Itera sobre las filas de media_edad_victimas
for index, row in media_edad_victimas.iterrows():
    # Extrae la víctima y la edad media de la fila actual
    victima = row['VICTIMA']
    edad_media = int(row['EDAD'])
   
    # Crea una máscara para las filas donde 'VICTIMA' coincide con la víctima actual
    mascara = df_victimas['VICTIMA'] == victima
    
    # Asigna la edad media a las filas que cumplen la condición
    df_victimas.loc[mascara, 'EDAD'] = np.where(df_victimas.loc[mascara, 'EDAD'].isnull(), edad_media, df_victimas.loc[mascara, 'EDAD'])

In [115]:
df_victimas[df_victimas['EDAD'].isnull()]

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO


In [116]:
df_victimas['EDAD'] = df_victimas['EDAD'].astype(int)#se cambia el tipo de dato a int


Para la columna "SEXO", se sustituirán los valores faltantes utilizando las modas correspondientes según el tipo de vehículo involucrado en el incidente.

In [117]:
# Filtra el df por los datos not null
df_victimas_filtro = df_victimas[df_victimas['SEXO'].notnull()]

In [118]:
moda_por_sexo = df_victimas_filtro.groupby('VICTIMA')['SEXO'].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan)


In [119]:
# Reemplazar los valores "SD" en 'sexo' con la moda correspondiente de 'victima'
for index, row in df_victimas.iterrows():
    if pd.isnull(row['SEXO']) and row['VICTIMA'] in moda_por_sexo.index:
        df_victimas.at[index, 'SEXO'] = moda_por_sexo[row['VICTIMA']]

In [120]:
df_victimas[df_victimas['SEXO'].isnull()]

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO


Verificamos la columna FECHA_FALLECIMIENTO y la comparamos con la fecha del accidente

In [121]:
df_victimas[df_victimas['FECHA_FALLECIMIENTO']==df_victimas['FECHA']]

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,2016-01-01,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01 00:00:00
1,2016-0002,2016-01-02,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70,2016-01-02 00:00:00
2,2016-0003,2016-01-03,2016,1,3,CONDUCTOR,MOTO,MASCULINO,30,2016-01-03 00:00:00
5,2016-0008,2016-01-24,2016,1,24,CONDUCTOR,MOTO,MASCULINO,30,2016-01-24 00:00:00
7,2016-0010,2016-01-29,2016,1,29,CONDUCTOR,MOTO,MASCULINO,18,2016-01-29 00:00:00
...,...,...,...,...,...,...,...,...,...,...
706,2021-0086,2021-11-25,2021,11,25,PEATON,PEATON,MASCULINO,75,2021-11-25 00:00:00
707,2021-0087,2021-11-27,2021,11,27,PEATON,PEATON,FEMENINO,73,2021-11-27 00:00:00
711,2021-0091,2021-12-11,2021,12,11,CONDUCTOR,MOTO,MASCULINO,24,2021-12-11 00:00:00
712,2021-0092,2021-12-12,2021,12,12,PEATON,PEATON,FEMENINO,50,2021-12-12 00:00:00


Se observa que en la mayoría de los casos, las víctimas fallecen en la misma fecha del incidente. Para los 68 casos donde no se disponen de datos, se tomará la fecha en la que ocurrió el accidente como referencia.

In [122]:
df_victimas['FECHA_FALLECIMIENTO'].fillna(df_victimas['FECHA'], inplace=True)# Reemplaza los valores NaN 

In [123]:
df_victimas['FECHA_FALLECIMIENTO'] = pd.to_datetime(df_victimas['FECHA_FALLECIMIENTO'])#Cambiar el tipo de la columna FECHA_FALLECIMIENTO

Para finalizar verificamos si quedan valores nulos

In [124]:
df_victimas.isnull().sum()

ID_hecho               0
FECHA                  0
AAAA                   0
MM                     0
DD                     0
ROL                    0
VICTIMA                0
SEXO                   0
EDAD                   0
FECHA_FALLECIMIENTO    0
dtype: int64

Se exporta el df como csv

In [125]:
df_victimas.to_csv('../datasets/victimas_clean.csv', index=False)

Se crea una tabla con la población estimada en CABA para los años del 2016 al 2021

In [126]:
# Definir los datos iniciales
poblacion_inicial = 2890151
poblacion_final = 3121707
tasa_crecimiento_anual = (poblacion_final / poblacion_inicial) ** (1 / (2022 - 2010)) - 1

In [127]:
# Crear una lista para almacenar los datos de la tabla
data = []

# Calcular la población estimada por semestre y agregar los datos a la lista
for year in range(2010, 2023):
    for semester in range(1, 3):  # 1 para primer semestre, 2 para segundo semestre
        poblacion_estimada = round(poblacion_inicial * (1 + tasa_crecimiento_anual) ** ((year - 2010) * 2 + semester - 1))
        data.append([f'{year}-S{semester}', poblacion_estimada])

# Crear el DataFrame a partir de la lista de datos
df = pd.DataFrame(data, columns=['Semestre', 'Población Estimada'])

In [128]:
# Mostrar el DataFrame
df

,Semestre,Población Estimada
0,2010-S1,2890151
1,2010-S2,2908773
2,2011-S1,2927515
3,2011-S2,2946378
4,2012-S1,2965362
5,2012-S2,2984469
6,2013-S1,3003698
7,2013-S2,3023052
8,2014-S1,3042530
9,2014-S2,3062134


In [129]:
# Seleccionar los semestres de los años 2016 a 2021
subset_df = df.loc[df['Semestre'].str[:4].astype(int).between(2016, 2021)]


In [130]:
subset_df

,Semestre,Población Estimada
12,2016-S1,3121707
13,2016-S2,3141821
14,2017-S1,3162065
15,2017-S2,3182439
16,2018-S1,3202944
17,2018-S2,3223581
18,2019-S1,3244352
19,2019-S2,3265256
20,2020-S1,3286295
21,2020-S2,3307469


In [131]:
subset_df.to_csv('../datasets/poblacion_estimada.csv', index=False)